##### Load libs:

In [2]:
import pandas as pd
import sqlite3

##### 1.Create connect:

In [3]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

##### 2.Create table "datamart":

In [53]:
connect.execute(
	"CREATE TABLE IF NOT EXISTS datamart AS " \
	"SELECT " \
		"c.uid, " \
		"c.labname, " \
		"c.timestamp AS first_commit_ts, " \
		"MIN(p.datetime) AS first_view_ts " \
	"FROM checker c " \
	"LEFT JOIN pageviews p ON p.uid=c.uid " \
	"WHERE " \
		"c.uid LIKE 'user_%' " \
		"AND c.status='ready' " \
		"AND c.numTrials=1 " \
		"AND c.labname IN ('laba04','laba04s','laba05','laba06','laba06s','project1') " \
	"GROUP BY c.uid, c.labname;"
)

CHECK: 

In [54]:
print(pd.io.sql.read_sql("SELECT * FROM datamart", connect))

        uid   labname             first_commit_ts               first_view_ts
0    user_1    laba04  2020-04-26 17:06:18.462708  2020-04-26 21:53:59.624136
1    user_1   laba04s  2020-04-26 17:12:11.843671  2020-04-26 21:53:59.624136
2    user_1    laba05  2020-05-02 19:15:18.540185  2020-04-26 21:53:59.624136
3    user_1    laba06  2020-05-17 16:26:35.268534  2020-04-26 21:53:59.624136
4    user_1   laba06s  2020-05-20 12:23:37.289724  2020-04-26 21:53:59.624136
..      ...       ...                         ...                         ...
135  user_8   laba04s  2020-04-19 10:22:35.761944                        None
136  user_8    laba05  2020-05-02 13:28:07.705193                        None
137  user_8    laba06  2020-05-16 17:56:15.755553                        None
138  user_8   laba06s  2020-05-16 20:01:07.900727                        None
139  user_8  project1  2020-05-14 15:42:04.002981                        None

[140 rows x 4 columns]


##### 3.Create DataFrames:

Create DataFrame "test":

In [ ]:
test=pd.io.sql.read_sql("SELECT * FROM datamart WHERE first_view_ts IS NOT NULL;",connect)
print(test)

        uid   labname             first_commit_ts               first_view_ts
0    user_1    laba04  2020-04-26 17:06:18.462708  2020-04-26 21:53:59.624136
1    user_1   laba04s  2020-04-26 17:12:11.843671  2020-04-26 21:53:59.624136
2    user_1    laba05  2020-05-02 19:15:18.540185  2020-04-26 21:53:59.624136
3    user_1    laba06  2020-05-17 16:26:35.268534  2020-04-26 21:53:59.624136
4    user_1   laba06s  2020-05-20 12:23:37.289724  2020-04-26 21:53:59.624136
5    user_1  project1  2020-05-14 20:56:08.898880  2020-04-26 21:53:59.624136
6   user_10    laba04  2020-04-25 08:24:52.696624  2020-04-18 12:19:50.182714
7   user_10   laba04s  2020-04-25 08:37:54.604222  2020-04-18 12:19:50.182714
8   user_10    laba05  2020-05-01 19:27:26.063245  2020-04-18 12:19:50.182714
9   user_10    laba06  2020-05-19 11:39:28.885637  2020-04-18 12:19:50.182714
10  user_10   laba06s  2020-05-20 07:37:31.175817  2020-04-18 12:19:50.182714
11  user_10  project1  2020-05-12 20:12:28.056618  2020-04-18 12

Create DataFrame "control":

In [60]:
control=pd.io.sql.read_sql("SELECT * FROM datamart WHERE first_view_ts IS NULL;",connect)
print(control)

        uid   labname             first_commit_ts first_view_ts
0   user_11    laba05  2020-05-03 21:06:55.970293          None
1   user_11  project1  2020-05-03 23:45:33.673409          None
2   user_12    laba04  2020-04-18 17:07:51.767358          None
3   user_12   laba04s  2020-04-26 15:42:38.070593          None
4   user_12    laba05  2020-05-03 08:39:25.174316          None
..      ...       ...                         ...           ...
76   user_8   laba04s  2020-04-19 10:22:35.761944          None
77   user_8    laba05  2020-05-02 13:28:07.705193          None
78   user_8    laba06  2020-05-16 17:56:15.755553          None
79   user_8   laba06s  2020-05-16 20:01:07.900727          None
80   user_8  project1  2020-05-14 15:42:04.002981          None

[81 rows x 4 columns]


In [68]:
mean_view_ts = pd.to_datetime(test['first_view_ts']).mean()
print(mean_view_ts)

2020-04-27 00:40:05.761783552


In [70]:
control['first_view_ts']=mean_view_ts

In [71]:
print(control)

        uid   labname             first_commit_ts  \
0   user_11    laba05  2020-05-03 21:06:55.970293   
1   user_11  project1  2020-05-03 23:45:33.673409   
2   user_12    laba04  2020-04-18 17:07:51.767358   
3   user_12   laba04s  2020-04-26 15:42:38.070593   
4   user_12    laba05  2020-05-03 08:39:25.174316   
..      ...       ...                         ...   
76   user_8   laba04s  2020-04-19 10:22:35.761944   
77   user_8    laba05  2020-05-02 13:28:07.705193   
78   user_8    laba06  2020-05-16 17:56:15.755553   
79   user_8   laba06s  2020-05-16 20:01:07.900727   
80   user_8  project1  2020-05-14 15:42:04.002981   

                   first_view_ts  
0  2020-04-27 00:40:05.761783552  
1  2020-04-27 00:40:05.761783552  
2  2020-04-27 00:40:05.761783552  
3  2020-04-27 00:40:05.761783552  
4  2020-04-27 00:40:05.761783552  
..                           ...  
76 2020-04-27 00:40:05.761783552  
77 2020-04-27 00:40:05.761783552  
78 2020-04-27 00:40:05.761783552  
79 2020-04-27

Save DataFrames:

In [72]:
test.to_sql('test',connect,if_exists='replace',index=False)
control.to_sql('control',connect,if_exists='replace',index=False)

81

CHECK:

In [ ]:
print(pd.io.sql.read_sql("SELECT * FROM test",connect))

        uid   labname             first_commit_ts               first_view_ts
0    user_1    laba04  2020-04-26 17:06:18.462708  2020-04-26 21:53:59.624136
1    user_1   laba04s  2020-04-26 17:12:11.843671  2020-04-26 21:53:59.624136
2    user_1    laba05  2020-05-02 19:15:18.540185  2020-04-26 21:53:59.624136
3    user_1    laba06  2020-05-17 16:26:35.268534  2020-04-26 21:53:59.624136
4    user_1   laba06s  2020-05-20 12:23:37.289724  2020-04-26 21:53:59.624136
5    user_1  project1  2020-05-14 20:56:08.898880  2020-04-26 21:53:59.624136
6   user_10    laba04  2020-04-25 08:24:52.696624  2020-04-18 12:19:50.182714
7   user_10   laba04s  2020-04-25 08:37:54.604222  2020-04-18 12:19:50.182714
8   user_10    laba05  2020-05-01 19:27:26.063245  2020-04-18 12:19:50.182714
9   user_10    laba06  2020-05-19 11:39:28.885637  2020-04-18 12:19:50.182714
10  user_10   laba06s  2020-05-20 07:37:31.175817  2020-04-18 12:19:50.182714
11  user_10  project1  2020-05-12 20:12:28.056618  2020-04-18 12

In [76]:
print(pd.io.sql.read_sql("SELECT * FROM control",connect))

        uid   labname             first_commit_ts               first_view_ts
0   user_11    laba05  2020-05-03 21:06:55.970293  2020-04-27 00:40:05.761783
1   user_11  project1  2020-05-03 23:45:33.673409  2020-04-27 00:40:05.761783
2   user_12    laba04  2020-04-18 17:07:51.767358  2020-04-27 00:40:05.761783
3   user_12   laba04s  2020-04-26 15:42:38.070593  2020-04-27 00:40:05.761783
4   user_12    laba05  2020-05-03 08:39:25.174316  2020-04-27 00:40:05.761783
..      ...       ...                         ...                         ...
76   user_8   laba04s  2020-04-19 10:22:35.761944  2020-04-27 00:40:05.761783
77   user_8    laba05  2020-05-02 13:28:07.705193  2020-04-27 00:40:05.761783
78   user_8    laba06  2020-05-16 17:56:15.755553  2020-04-27 00:40:05.761783
79   user_8   laba06s  2020-05-16 20:01:07.900727  2020-04-27 00:40:05.761783
80   user_8  project1  2020-05-14 15:42:04.002981  2020-04-27 00:40:05.761783

[81 rows x 4 columns]


##### 4. Close connect:

In [77]:
connect.close()